In [ ]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
from src.llms import get_llms, init_langchain, LLM
from src.experiment import get_fallacy_df, save_fallacy_df, run_experiment

# Load environment variables from .env file
load_dotenv()

init_langchain()

# Fallacy Identification Experiments

## Experiment 1: Fallacy Identification with zero-shot Prompt

In [ ]:
e1_filename = 'data/fallacies_e1.csv'
df_fallacies_e1 = get_fallacy_df(e1_filename)
df_fallacies_e1.head()

In [ ]:
llms = get_llms([LLM.CLAUDE_3_HAIKU])

prompt_template_e1 = "Is the following reasoning step correct? You can only answer \"Yes\" or \"No\".\n[step]"
run_experiment(df_fallacies_e1, e1_filename, prompt_template_e1, llms, sleep_seconds=0)

save_fallacy_df(df_fallacies_e1, e1_filename)

## Experiment 2: Fallacy Identification with few-shot Prompt

In [ ]:
e2_filename = 'data/fallacies_e2.csv'
df_fallacies_e2 = get_fallacy_df(e2_filename)
df_fallacies_e2.head()

In [7]:
llms = get_llms([LLM.GPT_4O, LLM.CLAUDE_3_5_SONNET, LLM.GEMINI_1_5_PRO])

prompt_template_e2 = """Is the following reasoning step correct? You can only answer \"Yes\" or \"No\".
Since if it's raining then the streets are wet and it's raining now, therefore, the streets are wet.
Yes.
Since I found a shell on the beach and this shell was beautifully shaped and colored, therefore, all shells are beautifully shaped and colored.
No.
Since I am at home or I am in the city and I am at home, therefore, I am not in the city.
No.
Since heavy snowfall often leads to traffic jams and traffic jams cause delays, therefore, heavy snowfall can lead to delays.
Yes.
[step]"""

run_experiment(df_fallacies_e2, e2_filename, prompt_template_e2, llms, sleep_seconds=0)

save_fallacy_df(df_fallacies_e2, e2_filename)


'Is the following reasoning step correct? You can only answer "Yes" or "No".\nSince if it\'s raining then the streets are wet and it\'s raining now, therefore, the streets are wet.\nYes.\nSince I found a shell on the beach and this shell was beautifully shaped and colored, therefore, all shells are beautifully shaped and colored.\nNo.\nSince I am at home or I am in the city and I am at home, therefore, I am not in the city.\nNo.\nSince heavy snowfall often leads to traffic jams and traffic jams cause delays, therefore, heavy snowfall can lead to delays.\nYes.\nSome question.'